# Managing population growth

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt
sys.path.append('..')
import helpers.dbquery as db
import helpers.functions as f
import yaml, ssl, asyncio, pickle, os, ast
import nb_black

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

In [2]:
# mapping to the modules that make the app

sys.path.insert(0, "../..")
sys.path.insert(0, "../../app")
sys.path.insert(0, "../../functions/popgrowth/")

from cmdb_graph import CosmosdbClient

c = CosmosdbClient()

In [3]:
params = yaml.safe_load(open(os.path.join(os.getenv("abspath"),"functions/popgrowth/settings.yml")))
syllables = pickle.load(open(os.path.join(os.getenv("abspath"),"app/creators/specs/syllables.p"), "rb"))
username = 'notebook'
params

{'pop_health_requirement': 0.6,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty']}

Time is updated by the `time` function. It is always running.

In [4]:
c.run_query("g.V().hasLabel('time').valueMap()")
params['time'] = c.clean_nodes(c.res)[0]
params['time']

{'currentTime': 28290,
 'updatedFrom': 'azfunction',
 'updatedAt': '2023-05-19 09:10:00.380788',
 'objid': '7263327370874',
 'name': 'time',
 'originTime': 0,
 'username': 'notebook',
 'objtype': 'time',
 'id': '7263327370874'}

For this notebook, setting the `pop_health_requirement` to an arbitrary value just to examine.

In [5]:
params['pop_health_requirement'] = 0.4

## Population Resource Consumption

In [6]:
from tools import consumption

We're going to run a resourse query test to ensure that consumption is happening. To test, update the `objid` with a planet that has consuming pops. 

In [7]:
validate_resources_updated_query = f"""
g.V().has('objid','6257988354925').out('has').valueMap()
"""
print(validate_resources_updated_query)
c.run_query(validate_resources_updated_query)
pre_consumption = pd.DataFrame.from_records(c.clean_nodes(c.res))
pre_consumption


g.V().has('objid','6257988354925').out('has').valueMap()



""


In [8]:

all_pops_query = f"""
g.V().has('label','pop').as('pop')
    .local(
        union(
            out('inhabits').as('location'),
            out('isOf').as('species')
            )
            .fold()).as('location','species')
        .path()
        .by(unfold().valueMap().fold())
"""

In [9]:
c.run_query(all_pops_query)
data = c.reduce_res(c.res)
pd.DataFrame(data)

,pop,location,species
0,"{'name': 'Shitianewlia Pecbia', 'objid': '1392...","{'name': 'Riepuerhy', 'class': 'terrestrial', ...","{'name': 'Gor', 'objid': '5400343991627', 'con..."
1,"{'name': 'Shitianewlia Hiaportcharlis', 'objid...","{'name': 'Riepuerhy', 'class': 'terrestrial', ...","{'name': 'Gor', 'objid': '5400343991627', 'con..."
2,"{'name': 'Shitianewlia Zi', 'objid': '08945611...","{'name': 'Riepuerhy', 'class': 'terrestrial', ...","{'name': 'Gor', 'objid': '5400343991627', 'con..."
3,"{'name': 'Woanggal Shakuwi', 'objid': '4097489...","{'name': 'Riepuerhy', 'class': 'terrestrial', ...","{'name': 'Gor', 'objid': '5400343991627', 'con..."
4,"{'name': 'Woanggal Lon', 'objid': '33564916986...","{'name': 'Riepuerhy', 'class': 'terrestrial', ...","{'name': 'Gor', 'objid': '5400343991627', 'con..."
5,"{'name': 'Woanggal Kas', 'objid': '17591583317...","{'name': 'Riepuerhy', 'class': 'terrestrial', ...","{'name': 'Gor', 'objid': '5400343991627', 'con..."
6,"{'name': 'Shitianewlia Gidongrang', 'objid': '...","{'name': 'Riepuerhy', 'class': 'terrestrial', ...","{'name': 'Gor', 'objid': '5400343991627', 'con..."
7,"{'health': 0.42, 'name': 'Shitianewlia Pecbiar...","{'name': 'Riepuerhy', 'class': 'terrestrial', ...","{'name': 'Gor', 'objid': '5400343991627', 'con..."
8,"{'health': 0.42, 'name': 'Shitianewlia Zicen',...","{'name': 'Riepuerhy', 'class': 'terrestrial', ...","{'name': 'Gor', 'objid': '5400343991627', 'con..."
9,"{'health': 0.42, 'name': 'Woanggal Shakuwigui'...","{'name': 'Riepuerhy', 'class': 'terrestrial', ...","{'name': 'Gor', 'objid': '5400343991627', 'con..."


In [10]:
pops_df = pd.DataFrame([d['pop'] for d in data])
pops_df

,name,objid,conformity,literacy,aggression,constitution,health,isIn,industry,wealth,factionLoyalty,isIdle,username,objtype,id
0,Shitianewlia Pecbia,1392740103248,0.606,0.265,0.652,0.394,0.70,6219407155821,0.5230,0.3940,0.554,true,BillmanLocal2,pop,1392740103248
1,Shitianewlia Hiaportcharlis,9922156506749,0.680,0.312,0.657,0.300,0.70,6219407155821,0.4785,0.3952,0.704,true,BillmanLocal2,pop,9922156506749
2,Shitianewlia Zi,0894561131236,0.622,0.360,0.588,0.311,0.70,6219407155821,0.4495,0.4048,0.596,true,BillmanLocal2,pop,0894561131236
3,Woanggal Shakuwi,4097489117670,0.678,0.457,0.515,0.418,0.70,3712738152632,0.4665,0.4618,0.592,true,BillmanLocal2,pop,4097489117670
4,Woanggal Lon,3356491698696,0.700,0.394,0.616,0.325,0.70,3712738152632,0.4705,0.4322,0.686,true,BillmanLocal2,pop,3356491698696
5,Woanggal Kas,1759158331704,0.709,0.455,0.661,0.351,0.70,3712738152632,0.5060,0.4805,0.723,true,BillmanLocal2,pop,1759158331704
6,Shitianewlia Gidongrang,3413748712870,0.624,0.374,0.541,0.292,0.70,6219407155821,0.4165,0.3952,0.713,true,BillmanLocal2,pop,3413748712870
7,Shitianewlia Pecbiarang,8953226925403,0.538,0.199,0.559,0.449,0.42,6219407155821,0.3140,0.2910,0.506,true,BillmanLocal2,pop,8953226925403
8,Shitianewlia Zicen,7199585478992,0.594,0.307,0.559,0.317,0.42,6219407155821,0.2700,0.1900,0.667,true,BillmanLocal2,pop,7199585478992
9,Woanggal Shakuwigui,6500296863192,0.628,0.552,0.520,0.384,0.42,3712738152632,0.2800,0.2910,0.516,true,BillmanLocal2,pop,6500296863192


In [11]:
species_df = pd.DataFrame([d['species'] for d in data])
species_df

,name,objid,consumes,effuses,viral_resilience,habitat_resilience,username,objtype,id
0,Gor,5400343991627,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,5400343991627
1,Gor,5400343991627,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,5400343991627
2,Gor,5400343991627,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,5400343991627
3,Gor,5400343991627,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,5400343991627
4,Gor,5400343991627,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,5400343991627
5,Gor,5400343991627,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,5400343991627
6,Gor,5400343991627,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,5400343991627
7,Gor,5400343991627,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,5400343991627
8,Gor,5400343991627,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,5400343991627
9,Gor,5400343991627,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,5400343991627


In [12]:
locations_df = pd.DataFrame([d['location'] for d in data])
locations_df

,name,class,objid,radius,mass,orbitsDistance,orbitsId,orbitsName,isSupportsLife,isPopulated,isHomeworld,username,objtype,id
0,Riepuerhy,terrestrial,9663052102446,0.065,0.827,1.490,7978035257500,Hin,true,true,true,BillmanLocal2,planet,9663052102446
1,Riepuerhy,terrestrial,9663052102446,0.065,0.827,1.490,7978035257500,Hin,true,true,true,BillmanLocal2,planet,9663052102446
2,Riepuerhy,terrestrial,9663052102446,0.065,0.827,1.490,7978035257500,Hin,true,true,true,BillmanLocal2,planet,9663052102446
3,Riepuerhy,terrestrial,9663052102446,0.065,0.827,1.490,7978035257500,Hin,true,true,true,BillmanLocal2,planet,9663052102446
4,Riepuerhy,terrestrial,9663052102446,0.065,0.827,1.490,7978035257500,Hin,true,true,true,BillmanLocal2,planet,9663052102446
5,Riepuerhy,terrestrial,9663052102446,0.065,0.827,1.490,7978035257500,Hin,true,true,true,BillmanLocal2,planet,9663052102446
6,Riepuerhy,terrestrial,9663052102446,0.065,0.827,1.490,7978035257500,Hin,true,true,true,BillmanLocal2,planet,9663052102446
7,Riepuerhy,terrestrial,9663052102446,0.065,0.827,1.490,7978035257500,Hin,true,true,true,BillmanLocal2,planet,9663052102446
8,Riepuerhy,terrestrial,9663052102446,0.065,0.827,1.490,7978035257500,Hin,true,true,true,BillmanLocal2,planet,9663052102446
9,Riepuerhy,terrestrial,9663052102446,0.065,0.827,1.490,7978035257500,Hin,true,true,true,BillmanLocal2,planet,9663052102446


First getting the list of consumption for each pop.


In [13]:
pops_df,species_df,locations_df = consumption.all_pops_consumption(c)


We expand that dataset to break open the list of people in each pop. 

In [14]:
params

{'pop_health_requirement': 0.4,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'time': {'currentTime': 28290,
  'updatedFrom': 'azfunction',
  'updatedAt': '2023-05-19 09:10:00.380788',
  'objid': '7263327370874',
  'name': 'time',
  'originTime': 0,
  'username': 'notebook',
  'objtype': 'time',
  'id': '7263327370874'}}

In [15]:
consumption_df = consumption.get_consumption_df(locations_df,species_df,params)
consumption_df = consumption.expand_consumption_df(consumption_df)
consumption_df

c:\Users\william.harding\anaconda3\envs\exoplanets\lib\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


,location_id,consumes,pop,consumption,multi
0,2793144513261,Organic,7,14,True
1,3493185958037,Organic,7,14,True
2,9663052102446,Organic,26,52,True


Then we get a list of the resources available on that location. 

In [16]:
c.run_query(consumption.make_resource_query(consumption_df))
resources = c.res
resources

[{'labels': [['location'], ['resource']],
  'objects': [{'objid': ['9663052102446'], 'name': ['Riepuerhy']},
   {'volume': [875], 'objid': ['9132171195260'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['2793144513261'], 'name': ['Townnak']},
   {'volume': [1011], 'objid': ['4741040123642'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['3493185958037'], 'name': ['Jivar']},
   {'volume': [1077], 'objid': ['4370165806739'], 'name': ['Organic']}]}]

In [17]:
consumption_df = consumption.tally_consumption(c,consumption_df,resources)
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,2793144513261,Organic,7,14,True,1011.0,997.0
1,3493185958037,Organic,7,14,True,1077.0,1063.0
2,9663052102446,Organic,26,52,True,875.0,823.0


Then we update the resources by taking out the resources that exist. You can check that this has happened in the graph by looking at the location ids. This is a verification query that isn't run in the function application.

In [18]:
consumption_df.apply(lambda x: consumption.make_resource_update_query(c,x),axis=1)

0    None
1    None
2    None
dtype: object

In [19]:
c.run_query(validate_resources_updated_query)
post_consumption = pd.DataFrame.from_records(c.clean_nodes(c.res))

In [20]:
pre_consumption

""


In [21]:
post_consumption

""


Now for the starving. Populations without enough food will starve (health declines). Populations with < 0 health will die.

In [22]:
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,2793144513261,Organic,7,14,True,1011.0,997.0
1,3493185958037,Organic,7,14,True,1077.0,1063.0
2,9663052102446,Organic,26,52,True,875.0,823.0


In [23]:
consumption_df[consumption_df['remaining']<=0].apply(lambda x: consumption.lower_health(c,params,x),axis=1)

,location_id,consumes,pop,consumption,multi,available,remaining


Looking more closely at `lower_health`
This only happens on locations that have < 0 resources. But here we're going to do it regardless

In [24]:
x = consumption_df.loc[0]
print(x)

dead_pop_nodes = []
dead_pop_ids = []
death_event_edges = []
query =f"""
g.V().has('objid','{x.location_id}').as('location').in('inhabits')
    .haslabel('pop').as('pop')
    .out('isOfSpecies').as('species')
    .path()
        .by(valueMap('objid','name'))
        .by(valueMap('name','objid','health','username'))
        .by(valueMap('name','objid','consumes'))
"""
c.run_query(query)
out = c.res
out

location_id    2793144513261
consumes             Organic
pop                        7
consumption               14
multi                   True
available             1011.0
remaining              997.0
Name: 0, dtype: object


[]

In [25]:
print(f"{len(out)} pops will starve in {x.location_id}")

0 pops will starve in 2793144513261


We are only going to starve the pop if the pop consumes that resource. 

health just goes down. 

Let's pretend that the pop will die. IF they do, it will create a `death event`.

In [26]:
death_event = consumption.death_by_starvation_event(c.clean_node(i['objects'][0]),i['objects'][1],params)
death_event

NameError: name 'i' is not defined

We will also link that event to an appropriate `happendAt` so that we can see it when looking at events related to specific things. 

In [ ]:
print(c.create_custom_edge(death_event, c.clean_node(i['objects'][0]), 'happenedAt').strip())

NameError: name 'death_event' is not defined

## Population growth
* Populations that die are deleted, so they don't grow.
* However all pops that can grow will, regardless of wheter they are starving or not. 
* Starving will lower thier health, which will eventually make them unable to grow. 


In [27]:
from tools import growth


In [28]:
pops_df,species_df,locations_df,factions_df = growth.get_pop_health(c,params)
print(f"lenth of pops == 0 : {len(pops_df)==0}")
pops_df

lenth of pops == 0 : False


,health,name,objid,conformity,literacy,aggression,constitution,isIn,industry,wealth,factionLoyalty,isIdle,username,objtype,id
0,0.7,Shitianewlia Pecbia,1392740103248,0.606,0.265,0.652,0.394,6219407155821,0.5230,0.3940,0.554,true,BillmanLocal2,pop,1392740103248
1,0.7,Shitianewlia Hiaportcharlis,9922156506749,0.680,0.312,0.657,0.300,6219407155821,0.4785,0.3952,0.704,true,BillmanLocal2,pop,9922156506749
2,0.7,Shitianewlia Zi,0894561131236,0.622,0.360,0.588,0.311,6219407155821,0.4495,0.4048,0.596,true,BillmanLocal2,pop,0894561131236
3,0.7,Woanggal Shakuwi,4097489117670,0.678,0.457,0.515,0.418,3712738152632,0.4665,0.4618,0.592,true,BillmanLocal2,pop,4097489117670
4,0.7,Woanggal Lon,3356491698696,0.700,0.394,0.616,0.325,3712738152632,0.4705,0.4322,0.686,true,BillmanLocal2,pop,3356491698696
5,0.7,Woanggal Kas,1759158331704,0.709,0.455,0.661,0.351,3712738152632,0.5060,0.4805,0.723,true,BillmanLocal2,pop,1759158331704
6,0.7,Shitianewlia Gidongrang,3413748712870,0.624,0.374,0.541,0.292,6219407155821,0.4165,0.3952,0.713,true,BillmanLocal2,pop,3413748712870
7,0.7,Bames Todin,4376681497460,0.408,0.469,0.474,0.380,3820091516344,0.4270,0.4480,0.373,true,Billmanserver,pop,4376681497460
8,0.7,Zokekegua Long,6644582427941,0.647,0.361,0.526,0.532,4353938110450,0.5290,0.4450,0.651,true,Billmanserver,pop,6644582427941
9,0.7,Bames Caslay,3983808991145,0.394,0.496,0.448,0.602,3820091516344,0.5250,0.5105,0.361,true,Billmanserver,pop,3983808991145


In [29]:
species_df

,name,objid,consumes,effuses,viral_resilience,habitat_resilience,username,objtype,id
0,Gor,5400343991627,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,5400343991627
1,Gor,5400343991627,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,5400343991627
2,Gor,5400343991627,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,5400343991627
3,Gor,5400343991627,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,5400343991627
4,Gor,5400343991627,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,5400343991627
5,Gor,5400343991627,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,5400343991627
6,Gor,5400343991627,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,5400343991627
7,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
8,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
9,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536


In [30]:
locations_df

,name,class,objid,radius,mass,orbitsDistance,orbitsId,orbitsName,isSupportsLife,isPopulated,isHomeworld,username,objtype,id
0,Riepuerhy,terrestrial,9663052102446,0.065,0.827,1.490,7978035257500,Hin,true,true,true,BillmanLocal2,planet,9663052102446
1,Riepuerhy,terrestrial,9663052102446,0.065,0.827,1.490,7978035257500,Hin,true,true,true,BillmanLocal2,planet,9663052102446
2,Riepuerhy,terrestrial,9663052102446,0.065,0.827,1.490,7978035257500,Hin,true,true,true,BillmanLocal2,planet,9663052102446
3,Riepuerhy,terrestrial,9663052102446,0.065,0.827,1.490,7978035257500,Hin,true,true,true,BillmanLocal2,planet,9663052102446
4,Riepuerhy,terrestrial,9663052102446,0.065,0.827,1.490,7978035257500,Hin,true,true,true,BillmanLocal2,planet,9663052102446
5,Riepuerhy,terrestrial,9663052102446,0.065,0.827,1.490,7978035257500,Hin,true,true,true,BillmanLocal2,planet,9663052102446
6,Riepuerhy,terrestrial,9663052102446,0.065,0.827,1.490,7978035257500,Hin,true,true,true,BillmanLocal2,planet,9663052102446
7,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
8,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
9,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261


In [31]:
factions_df

,name,objid,lat,long,pop_loactions,username,objtype,id
0,Shitianewlia,6219407155821,-0.066,0.0,"[[0, 0], [2, -2], [3, -3], [-2, 0], [1, -1]]",BillmanLocal2,faction,6219407155821
1,Shitianewlia,6219407155821,-0.066,0.0,"[[0, 0], [2, -2], [3, -3], [-2, 0], [1, -1]]",BillmanLocal2,faction,6219407155821
2,Shitianewlia,6219407155821,-0.066,0.0,"[[0, 0], [2, -2], [3, -3], [-2, 0], [1, -1]]",BillmanLocal2,faction,6219407155821
3,Woanggal,3712738152632,0.066,-0.0,"[[0, 0], [-1, 1], [-2, 2], [0, -2]]",BillmanLocal2,faction,3712738152632
4,Woanggal,3712738152632,0.066,-0.0,"[[0, 0], [-1, 1], [-2, 2], [0, -2]]",BillmanLocal2,faction,3712738152632
5,Woanggal,3712738152632,0.066,-0.0,"[[0, 0], [-1, 1], [-2, 2], [0, -2]]",BillmanLocal2,faction,3712738152632
6,Shitianewlia,6219407155821,-0.066,0.0,"[[0, 0], [2, -2], [3, -3], [-2, 0], [1, -1]]",BillmanLocal2,faction,6219407155821
7,Bames,3820091516344,0.105,0.0,"[[0, 0], [-1, -1], [-2, 0], [0, 2], [-1, 1]]",Billmanserver,faction,3820091516344
8,Zokekegua,4353938110450,-0.105,0.0,"[[0, 0], [2, 0], [-2, 0], [0, -2]]",Billmanserver,faction,4353938110450
9,Bames,3820091516344,0.105,0.0,"[[0, 0], [-1, -1], [-2, 0], [0, 2], [-1, 1]]",Billmanserver,faction,3820091516344


In [32]:
factions_df['pop_locations']

KeyError: 'pop_locations'

In [31]:
pops_df['roll'] = pops_df['objid'].apply(lambda x: np.random.random())
pops_df['grow'] = pops_df[['wealth','health']].T.mean() >= pops_df['roll']

reproducing_pops = pops_df[pops_df['grow']].drop(['roll','grow'],axis=1)
pops_df

,health,name,objid,conformity,literacy,aggression,constitution,isIn,industry,wealth,factionLoyalty,isIdle,username,objtype,id,isInFaction,roll,grow
0,0.70,Mac Boven,0374796045500,0.627,0.525,0.327,0.494,1066241530501,0.4105,0.4678,0.659,true,BillmanLocal2,pop,0374796045500,NaN,0.680375,False
1,0.70,Mac Serdasgasre,8410409646694,0.575,0.534,0.547,0.481,1066241530501,0.5140,0.5240,0.640,true,BillmanLocal2,pop,8410409646694,NaN,0.724255,False
2,0.70,Pest Rangpizeray,1438093180356,0.245,0.376,0.501,0.416,8779704672507,0.4585,0.4172,0.116,true,BillmanLocal2,pop,1438093180356,NaN,0.954751,False
3,0.70,Mac Latos,8673992196155,0.409,0.671,0.614,0.468,1066241530501,0.5410,0.6060,0.595,true,BillmanLocal2,pop,8673992196155,NaN,0.793990,False
4,0.70,Mac Fer,5225645311508,0.491,0.374,0.466,0.551,1066241530501,0.5085,0.4412,0.418,true,BillmanLocal2,pop,5225645311508,NaN,0.293308,True
5,0.70,Mac Guerisgaonlin,9525508045282,0.447,0.482,0.492,0.620,1066241530501,0.5560,0.5190,0.401,true,BillmanLocal2,pop,9525508045282,NaN,0.595234,True
6,0.70,Mac Lipet,3539278056453,0.361,0.626,0.396,0.505,1066241530501,0.4505,0.5382,0.367,true,BillmanLocal2,pop,3539278056453,NaN,0.364523,True
7,0.42,Mac Serdasgasrenam,5022325449201,0.617,0.580,0.602,0.521,1066241530501,0.3080,0.2480,0.729,true,BillmanLocal2,pop,5022325449201,NaN,0.542197,False
8,0.42,Mac Ferwood,4915654495902,0.454,0.319,0.539,0.545,1066241530501,0.3050,0.3350,0.476,true,BillmanLocal2,pop,4915654495902,NaN,0.234161,True
9,0.42,Mac Lipetbat,0697143188419,0.446,0.725,0.317,0.495,1066241530501,0.2700,0.3810,0.356,true,BillmanLocal2,pop,0697143188419,NaN,0.024401,True


In [32]:
reproducing_pops

,health,name,objid,conformity,literacy,aggression,constitution,isIn,industry,wealth,factionLoyalty,isIdle,username,objtype,id,isInFaction
4,0.70,Mac Fer,5225645311508,0.491,0.374,0.466,0.551,1066241530501,0.5085,0.4412,0.418,true,BillmanLocal2,pop,5225645311508,NaN
5,0.70,Mac Guerisgaonlin,9525508045282,0.447,0.482,0.492,0.620,1066241530501,0.5560,0.5190,0.401,true,BillmanLocal2,pop,9525508045282,NaN
6,0.70,Mac Lipet,3539278056453,0.361,0.626,0.396,0.505,1066241530501,0.4505,0.5382,0.367,true,BillmanLocal2,pop,3539278056453,NaN
8,0.42,Mac Ferwood,4915654495902,0.454,0.319,0.539,0.545,1066241530501,0.3050,0.3350,0.476,true,BillmanLocal2,pop,4915654495902,NaN
9,0.42,Mac Lipetbat,0697143188419,0.446,0.725,0.317,0.495,1066241530501,0.2700,0.3810,0.356,true,BillmanLocal2,pop,0697143188419,NaN
13,0.42,Mac Lipetper,2829371972883,0.381,0.583,0.470,0.444,1066241530501,0.2700,0.3370,0.363,true,BillmanLocal2,pop,2829371972883,NaN
16,0.50,Kyma Torhia,6086380270986,0.650,0.643,0.609,0.445,NaN,0.5270,0.5850,0.790,true,Billmanserver,pop,6086380270986,3492720234201
17,0.50,Kyma Matanyelan,6996212056531,0.489,0.553,0.635,0.515,NaN,0.5750,0.5640,0.413,true,Billmanserver,pop,6996212056531,3492720234201
18,0.50,Cen Prohinmacnew,6665457894547,0.705,0.398,0.554,0.622,NaN,0.5880,0.4930,0.633,true,Billmanserver,pop,6665457894547,6678260056295
20,0.50,Kyma Ga,1335426076073,0.484,0.576,0.482,0.507,NaN,0.4945,0.5352,0.584,true,Billmanserver,pop,1335426076073,3492720234201


In [33]:
nodes = []
edges = []
# Event edges must be uploaded separately, because the event nodes haven't been created yet.    
event_edges = []

In [34]:
for i in reproducing_pops.index.to_list():
    p = reproducing_pops.loc[i].to_dict()
    species = species_df.loc[i].to_dict()
    location = locations_df.loc[i].to_dict()
    child = growth.grow_pop(p,species,params,syllables)
    nodes.append(child)
    event = growth.population_growth_event(p, location,child,params)
    nodes.append(event)
    edges.append({"node1": child["objid"], "node2": p["objid"], "label": "childOf"})
    edges.append({"node1": child["objid"], "node2": child["isIn"], "label": "isIn"})
    edges.append({"node1": child["objid"], "node2": species["objid"], "label": "isOf"})
    event_edges.append(c.create_custom_edge(event,location,'happenedAt'))
    event_edges.append(c.create_custom_edge(p,event,'caused'))



In [35]:
p

{'health': 0.42,
 'name': 'Mac Guerisgaonlinros',
 'objid': '0323461375645',
 'conformity': 0.356,
 'literacy': 0.404,
 'aggression': 0.417,
 'constitution': 0.546,
 'isIn': '1066241530501',
 'industry': 0.334,
 'wealth': 0.318,
 'factionLoyalty': 0.339,
 'isIdle': 'true',
 'username': 'BillmanLocal2',
 'objtype': 'pop',
 'id': '0323461375645',
 'isInFaction': nan}

In [36]:
growth.grow_pop(p,species,params,syllables)

{'health': 0.252,
 'name': 'Mac Guerisgaonlinrosho',
 'objid': '8361218224320',
 'conformity': 0.299,
 'literacy': 0.317,
 'aggression': 0.323,
 'constitution': 0.532,
 'isIn': '1066241530501',
 'industry': 0.2,
 'wealth': 0.126,
 'factionLoyalty': 0.387,
 'isIdle': 'true',
 'username': 'BillmanLocal2',
 'objtype': 'pop',
 'id': '8361218224320',
 'isInFaction': nan,
 'label': 'pop'}

In [37]:
pd.DataFrame.from_records(nodes)


,health,name,objid,conformity,literacy,aggression,constitution,isIn,industry,wealth,factionLoyalty,isIdle,username,objtype,id,isInFaction,label,text,visibleTo,time
0,0.420,Mac Ferga,0287502876706,0.557,0.428,0.451,0.615,1066241530501,0.305,0.351,0.374,true,BillmanLocal2,pop,0287502876706,NaN,pop,NaN,NaN,NaN
1,NaN,population growth,2464638614452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,event,NaN,NaN,NaN,event,The population (Mac Fer) inhabiting Gunmejeflo...,BillmanLocal2,28277.0
2,0.420,Mac Guerisgaonlinfon,3894182758189,0.475,0.447,0.462,0.668,1066241530501,0.334,0.396,0.464,true,BillmanLocal2,pop,3894182758189,NaN,pop,NaN,NaN,NaN
3,NaN,population growth,4122996995167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,event,NaN,NaN,NaN,event,The population (Mac Guerisgaonlin) inhabiting ...,BillmanLocal2,28277.0
4,0.420,Mac Lipetam,0887781405662,0.322,0.581,0.340,0.561,1066241530501,0.270,0.296,0.278,true,BillmanLocal2,pop,0887781405662,NaN,pop,NaN,NaN,NaN
5,NaN,population growth,9315204094813,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,event,NaN,NaN,NaN,event,The population (Mac Lipet) inhabiting Gunmejef...,BillmanLocal2,28277.0
6,0.252,Mac Ferwooder,8477282992499,0.390,0.231,0.442,0.450,1066241530501,0.183,0.137,0.571,true,BillmanLocal2,pop,8477282992499,NaN,pop,NaN,NaN,NaN
7,NaN,population growth,5552970289799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,event,NaN,NaN,NaN,event,The population (Mac Ferwood) inhabiting Gunmej...,BillmanLocal2,28277.0
8,0.252,Mac Lipetbatxi,5491748216576,0.414,0.714,0.298,0.520,1066241530501,0.162,0.319,0.275,true,BillmanLocal2,pop,5491748216576,NaN,pop,NaN,NaN,NaN
9,NaN,population growth,2366653475789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,event,NaN,NaN,NaN,event,The population (Mac Lipetbat) inhabiting Gunme...,BillmanLocal2,28277.0


In [38]:
pd.DataFrame.from_records(edges)

,node1,node2,label
0,0287502876706,5225645311508,childOf
1,0287502876706,1066241530501,isIn
2,0287502876706,8132528530336,isOf
3,3894182758189,9525508045282,childOf
4,3894182758189,1066241530501,isIn
5,3894182758189,8132528530336,isOf
6,0887781405662,3539278056453,childOf
7,0887781405662,1066241530501,isIn
8,0887781405662,8132528530336,isOf
9,8477282992499,4915654495902,childOf


In [39]:
event_edges

["\n        g.V().has('objid','2464638614452')\n            .addE('happenedAt')\n            .to(g.V().has('objid','7946732117793'))\n        ",
 "\n        g.V().has('objid','5225645311508')\n            .addE('caused')\n            .to(g.V().has('objid','2464638614452'))\n        ",
 "\n        g.V().has('objid','4122996995167')\n            .addE('happenedAt')\n            .to(g.V().has('objid','7946732117793'))\n        ",
 "\n        g.V().has('objid','9525508045282')\n            .addE('caused')\n            .to(g.V().has('objid','4122996995167'))\n        ",
 "\n        g.V().has('objid','9315204094813')\n            .addE('happenedAt')\n            .to(g.V().has('objid','7946732117793'))\n        ",
 "\n        g.V().has('objid','3539278056453')\n            .addE('caused')\n            .to(g.V().has('objid','9315204094813'))\n        ",
 "\n        g.V().has('objid','5552970289799')\n            .addE('happenedAt')\n            .to(g.V().has('objid','7946732117793'))\n        

In [40]:
growth.grow(c,params,syllables)

GremlinServerError: 597: 

ActivityId : 10f79a05-8a33-44c0-8ba2-c471828ef52e
ExceptionType : GraphCompileException
ExceptionMessage : 
	Gremlin Query Compilation Error: Unable to resolve symbol 'nan' in the current context. @ line 1, column 488.
	1 Error(s)
	GremlinRequestId : b01bd939-7414-4f93-aa9d-0004f388ed5b
	Context : graphcompute
	Scope : graphparse-translate-validatesymbolresolution
	GraphInterOpStatusCode : QuerySyntaxError
	HResult : 0x80131500
